In [4]:
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
from anndata import AnnData
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import seaborn as sns
import tangram as tg
import os
import random
import os
import numpy as np
import torch

import pandas as pd
from scipy import stats
import scipy.stats as st
import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt
import os
import torch
from torch.nn.functional import softmax, cosine_similarity, sigmoid
import tangram as tg

from utils import CalculateMeteics

In [51]:
def Tangram_impute(K, RNA_data_adata1, Spatial_data_adata1, train_gene, predict_gene,
    annotate = None, modes = 'clusters', density = 'rna_count_based'):

    print ('We run Tangram for data')
    test_list = predict_gene[K]
    test_list = [x.lower() for x in test_list]
    train_list = train_gene[K]
    spatial_data_partial = Spatial_data_adata1[:, train_list]
    train_list = np.array(train_list)
    if annotate == None:
        RNA_data_adata_label = RNA_data_adata1
        sc.pp.normalize_total(RNA_data_adata_label)
        sc.pp.log1p(RNA_data_adata_label)
        sc.pp.highly_variable_genes(RNA_data_adata_label)
        RNA_data_adata_label = RNA_data_adata_label[:, RNA_data_adata_label.var.highly_variable]
        sc.pp.scale(RNA_data_adata_label, max_value=10)
        sc.tl.pca(RNA_data_adata_label)
        sc.pp.neighbors(RNA_data_adata_label)
        sc.tl.leiden(RNA_data_adata_label, resolution = 0.5)
        RNA_data_adata.obs['leiden']  = RNA_data_adata_label.obs.leiden
    else:
        global CellTypeAnnotate
        RNA_data_adata1.obs['leiden']  = CellTypeAnnotate
    tg.pp_adatas(RNA_data_adata1, spatial_data_partial, genes=train_list)
    device = torch.device('cuda:0')
    if modes == 'clusters':
        ad_map = tg.map_cells_to_space(RNA_data_adata1, spatial_data_partial, device = device, mode = modes, cluster_label = 'leiden', density_prior = density)
        ad_ge = tg.project_genes(ad_map, RNA_data_adata1, cluster_label = 'leiden')
    else:
        ad_map = tg.map_cells_to_space(RNA_data_adata, spatial_data_partial, device = device, mode=modes, target_count=spatial_data_partial.shape[1], lambda_f_reg=1, lambda_count=1)
        ad_ge = tg.project_genes(ad_map, RNA_data_adata1)
    test_list = list(set(ad_ge.var_names) & set(test_list))
    test_list = np.array(test_list)
    pre_gene = pd.DataFrame(ad_ge[:,test_list].X, index=ad_ge[:,test_list].obs_names, columns=ad_ge[:,test_list].var_names)
    all_gene = pd.DataFrame(ad_ge.X, index=ad_ge.obs_names, columns=ad_ge.var_names)
    return pre_gene, all_gene


In [52]:
def Tangram_impute2(K, RNA_data_adata1, Spatial_data_adata1, train_gene, predict_gene,
    annotate = None, modes = 'constrained', density = 'rna_count_based'):

    print ('We run Tangram for data')
    test_list = predict_gene[K]
    test_list = [x.lower() for x in test_list]
    train_list = train_gene[K]
    spatial_data_partial = Spatial_data_adata1[:, train_list]
    train_list = np.array(train_list)
    if annotate == None:
        RNA_data_adata_label = RNA_data_adata1
        sc.pp.normalize_total(RNA_data_adata_label)
        sc.pp.log1p(RNA_data_adata_label)
        sc.pp.highly_variable_genes(RNA_data_adata_label)
        RNA_data_adata_label = RNA_data_adata_label[:, RNA_data_adata_label.var.highly_variable]
        sc.pp.scale(RNA_data_adata_label, max_value=10)
        sc.tl.pca(RNA_data_adata_label)
        sc.pp.neighbors(RNA_data_adata_label)
        sc.tl.leiden(RNA_data_adata_label, resolution = 0.5)
        RNA_data_adata.obs['leiden']  = RNA_data_adata_label.obs.leiden
    else:
        global CellTypeAnnotate
        RNA_data_adata1.obs['leiden']  = CellTypeAnnotate
    tg.pp_adatas(RNA_data_adata1, spatial_data_partial, genes=train_list)
    device = torch.device('cuda:0')
    if modes == 'clusters':
        ad_map = tg.map_cells_to_space(RNA_data_adata1, spatial_data_partial, device = device, mode = modes, cluster_label = 'leiden', density_prior = density)
        ad_ge = tg.project_genes(ad_map, RNA_data_adata1, cluster_label = 'leiden')
    else:
        ad_map = tg.map_cells_to_space(RNA_data_adata, spatial_data_partial, device = device, mode=modes, target_count=spatial_data_partial.shape[1], lambda_f_reg=1, lambda_count=1)
        ad_ge = tg.project_genes(ad_map, RNA_data_adata1)
    test_list = list(set(ad_ge.var_names) & set(test_list))
    test_list = np.array(test_list)
    pre_gene = pd.DataFrame(ad_ge[:,test_list].X, index=ad_ge[:,test_list].obs_names, columns=ad_ge[:,test_list].var_names)
    all_gene = pd.DataFrame(ad_ge.X, index=ad_ge.obs_names, columns=ad_ge.var_names)
    return pre_gene, all_gene


In [7]:
DataDir = '../dataset/benchmark_datasets/Dataset4/'
outdir = '../results/benchmark_datasets/Dataset4'
if not os.path.exists(outdir):
    os.makedirs(outdir)

RNA_file = DataDir + 'scRNA_count.h5ad'
Spatial_file = DataDir + 'Insitu_count.h5ad'
location_file = DataDir + 'Locations.txt'

# RNA_data = pd.read_table(RNA_file, header=0, index_col = 0)
# Spatial_data = pd.read_table(Spatial_file, sep = '\t',header = 0)
RNA_data_adata = sc.read(RNA_file)
Spatial_data_adata = sc.read(Spatial_file)
# locations = np.loadtxt(location_file, skiprows=1)

train_gene = np.load(DataDir + 'train_list.npy', allow_pickle=True).tolist()
predict_gene = np.load(DataDir+'test_list.npy', allow_pickle=True).tolist()
device = 'GPU'

In [8]:
result, all_results = Tangram_impute(0, RNA_data_adata, Spatial_data_adata, train_gene, predict_gene)

We run Tangram for data


INFO:root:900 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:900 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 900 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.548, KL reg: 0.249
Score: 0.750, KL reg: 0.000
Score: 0.752, KL reg: 0.000
Score: 0.752, KL reg: 0.000
Score: 0.753, KL reg: 0.000
Score: 0.753, KL reg: 0.000
Score: 0.753, KL reg: 0.000
Score: 0.753, KL reg: 0.000
Score: 0.753, KL reg: 0.000
Score: 0.753, KL reg: 0.000


INFO:root:Saving results..


In [32]:
test_list = predict_gene[0]
test_list = [x.lower() for x in test_list]

result.columns = [x.lower() for x in result.columns]
result_test = result[test_list]
# predict = Spatial_data_adata[:, test_list]
df = pd.DataFrame(Spatial_data_adata.X, Spatial_data_adata.obs.index, Spatial_data_adata.var.index)
df.columns = [x.lower() for x in df.columns]
df_test = df[test_list]


In [33]:
prefix = '../results/benchmark_datasets/Dataset4/'
metric = 'all'
name = 'tangram'
# predict = Spatial_data_adata[:, result.columns]
# df = pd.DataFrame(predict.X, predict.obs.index, predict.var.index)

evaluate = CalculateMeteics(df_test, result_test, prefix, metric, name)
acc = evaluate.compute_all()

(524, 100) (524, 100)
SSIM
PCC
JS
RMSE


In [35]:
acc.T['PCC'].mean()

0.3724415915018319

In [42]:
result2, all_results2 = Tangram_impute2(0, RNA_data_adata, Spatial_data_adata, train_gene, predict_gene)

We run Tangram for data


INFO:root:900 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:900 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 900 genes and rna_count_based density_prior in constrained mode...


Score: 0.599, KL reg: 0.187, Count reg: 6217.080, Lambda f reg: 2940.947
Score: 0.790, KL reg: 0.000, Count reg: 343.977, Lambda f reg: 421.716
Score: 0.792, KL reg: 0.000, Count reg: 1.038, Lambda f reg: 351.760
Score: 0.791, KL reg: 0.000, Count reg: 1.528, Lambda f reg: 219.353
Score: 0.790, KL reg: 0.000, Count reg: 3.278, Lambda f reg: 167.707
Score: 0.790, KL reg: 0.000, Count reg: 0.208, Lambda f reg: 143.800
Score: 0.790, KL reg: 0.000, Count reg: 1.441, Lambda f reg: 125.378
Score: 0.790, KL reg: 0.000, Count reg: 1.612, Lambda f reg: 111.371
Score: 0.790, KL reg: 0.000, Count reg: 0.496, Lambda f reg: 102.521
Score: 0.790, KL reg: 0.000, Count reg: 0.720, Lambda f reg: 96.279


INFO:root:Saving results..


In [43]:
test_list = predict_gene[0]
test_list = [x.lower() for x in test_list]

result.columns = [x.lower() for x in result.columns]
result_test = result2[test_list]
# predict = Spatial_data_adata[:, test_list]
df = pd.DataFrame(Spatial_data_adata.X, Spatial_data_adata.obs.index, Spatial_data_adata.var.index)
df.columns = [x.lower() for x in df.columns]
df_test = df[test_list]

prefix = '../results/benchmark_datasets/Dataset4/'
metric = 'all'
name = 'tangram'
# predict = Spatial_data_adata[:, result.columns]
# df = pd.DataFrame(predict.X, predict.obs.index, predict.var.index)

evaluate = CalculateMeteics(df_test, result_test, prefix, metric, name)
acc = evaluate.compute_all()

(524, 100) (524, 100)
SSIM
PCC
JS
RMSE


In [44]:
acc.T['PCC'].mean()

0.34067098232077275

In [49]:
result, all_results = Tangram_impute(9, RNA_data_adata, Spatial_data_adata, train_gene, predict_gene)

We run Tangram for data


INFO:root:900 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:900 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 900 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.556, KL reg: 0.223
Score: 0.728, KL reg: 0.000
Score: 0.730, KL reg: 0.000
Score: 0.730, KL reg: 0.000
Score: 0.731, KL reg: 0.000
Score: 0.731, KL reg: 0.000
Score: 0.731, KL reg: 0.000
Score: 0.731, KL reg: 0.000
Score: 0.731, KL reg: 0.000
Score: 0.731, KL reg: 0.000


INFO:root:Saving results..


In [54]:
test_list = predict_gene[9]
test_list = [x.lower() for x in test_list]

result.columns = [x.lower() for x in result.columns]
result_test = result[test_list]
# predict = Spatial_data_adata[:, test_list]
df = pd.DataFrame(Spatial_data_adata.X, Spatial_data_adata.obs.index, Spatial_data_adata.var.index)
df.columns = [x.lower() for x in df.columns]
df_test = df[test_list]

prefix = '../results/benchmark_datasets/Dataset4/'
metric = 'all'
name = 'tangram'
# predict = Spatial_data_adata[:, result.columns]
# df = pd.DataFrame(predict.X, predict.obs.index, predict.var.index)

evaluate = CalculateMeteics(df_test, result_test, prefix, metric, name)
acc = evaluate.compute_all()

(524, 100) (524, 100)
SSIM
PCC
JS
RMSE


In [58]:
acc.T['PCC']

arhgap32    0.806117
nptx1       0.574428
atp6v0a1    0.840822
actr1b      0.693068
abhd8       0.753320
              ...   
cldn11      0.913741
gsn         0.265707
cnp         0.279282
cplx1       0.768569
mag         0.866570
Name: PCC, Length: 100, dtype: float64

In [56]:
df_test['cldn11']

0      0.0
1      2.0
2      0.0
3      0.0
4      0.0
      ... 
519    0.0
520    0.0
521    0.0
522    4.0
523    0.0
Name: cldn11, Length: 524, dtype: float32

In [57]:
result_test['cldn11']

0      0.112455
1      0.085404
2      0.088387
3      0.048252
4      0.024804
         ...   
519    0.073114
520    0.145349
521    0.224267
522    0.198171
523    0.003126
Name: cldn11, Length: 524, dtype: float32